# Toy parton shower
We are going to simulate the branching of a high-energy quark. This is an oversimplified example in which only the leading parton branches and we keep track of just the transverse momentum of the emission. Further, energy-momentum conservation is not taken into account.

In [1]:
from random import random
from math import pi, exp, log, sqrt
import matplotlib.pyplot as plt
from ipywidgets import interactive
import numpy as np

# value of the strong coupling
alphas = 0.12
# choose a quark as the initiator 
CF=4/3

np.random.seed(42) # Set the random number generator to a fixed sequence

# First we define the probability of no-emission (Sudakov form factor)
def ptFromSudakov(sudakovValue,ptHigh):
    """Returns the pt value that solves the relation 
       Sudakov = sudakovValue (for 0 < sudakovValue < 1)
    """
    norm = (2*CF/pi)
    # r = Sudakov = exp(-alphas * norm * L^2)
    # --> log(r) = -alphas * norm * L^2
    # --> L^2 = log(r)/(-alphas*norm)
    L2 = log(sudakovValue)/(-alphas * norm)
    pt = ptHigh * exp(-sqrt(L2))
    return pt

# Now we generate an "event" 
def event(ptHigh,ptCut):
    # start with maximum possible value of Sudakov
    sudakov  = 1
    pt_values = []
    while (True):
        # scale it by a random number 
        sudakov *= random()
        # deduce the corresponding pt
        pt = ptFromSudakov(sudakov,ptHigh)
        # if pt falls below the cutoff, event is finished
        if (pt < ptCut): 
            break
        #else, add it to the "event record"
        else:
            pt_values.append(pt)
    return pt_values

# And do some event loop + plotting
def main(nev, ptHigh, ptCut):
    particle_list = []
    for iev in range(0,nev):
        particle_list = particle_list+event(ptHigh,ptCut)
    particles = np.array(particle_list,dtype=object)
    nem = np.size(particles)
    # make a little plot
    plt.xlabel(r"$p_t$ (GeV)",fontsize=20)
    plt.ylabel(r"$N_{\rm em}$",fontsize=20)
    plt.yscale('log')
    plt.title(fr'$N_{{\rm{{ev}}}}={nev}$, '
            fr'$p_{{t,\rm{{max}}}}={ptHigh}$ GeV, '
            fr'$p_{{t,\rm{{cut}}}}={ptCut}$ GeV, '
            fr'$N_{{\rm{{em}}}}={nem}$')
    pt_bins = np.arange(0,max(particles),1)
    
    plt.hist(particles, bins=pt_bins)
    print("average pt", np.mean(particles))


interactive_main = interactive(main, nev=(0,1000,100), ptHigh=(15,100,5), ptCut=(1,4,1))
interactive_main
        

interactive(children=(IntSlider(value=500, description='nev', max=1000, step=100), IntSlider(value=55, descrip…

Now we are going to smear the previous distribution by including transverse momentum broadening sampled from a Gaussian with average $\hat q L$, with $\hat q$ being the quenching parameter and $L$ the length of the medium.

In [2]:
GeVtoFm = 0.2 # convert units
ptHigh = 100
ptCut = 2

def broadening(qhat,L):
    qhat *= GeVtoFm # [GeV^3]
    L /= GeVtoFm #[GeV^-1]
    sigma = np.sqrt(qhat*L)
    delta_kt = -1
    # Generate a positive kt 
    while(delta_kt<0):
        delta_kt = np.random.normal(0, sigma)
    return delta_kt

# Create shower + broadening 
def event_wbroadening(qhat,L):
    # start with maximum possible value of Sudakov
    sudakov  = 1
    pt_values = []
    while (True):
        # scale it by a random number 
        sudakov *= random()
        # deduce the corresponding pt
        pt = ptFromSudakov(sudakov,ptHigh)
        # if pt falls below the cutoff, event is finished
        if (pt < ptCut): 
            break
        #else, add it to the "event record"
        else:
            pt_values.append(pt+broadening(qhat,L))
    return pt_values

# And do some event loop + plotting
def main_wbroad(nev, qhat, L):
    particle_list = []
    for iev in range(0,nev):
        particle_list = particle_list+event_wbroadening(qhat,L)
    particles = np.array(particle_list,dtype=object)
    nem = np.size(particles)
    # make a little plot
    plt.xlabel(r"$p_t$ (GeV)",fontsize=20)
    plt.ylabel(r"$N_{\rm em}$",fontsize=20)
    plt.yscale('log')
    plt.title(fr'$N_{{\rm{{ev}}}}={nev}$, '
            fr'$\hat q={qhat}$ GeV$^2$/fm, '
            fr'$L={L}$ fm, '
            fr'$N_{{\rm{{em}}}}={nem}$')
    pt_bins = np.arange(0,max(particles),1)
    plt.hist(particles, bins=pt_bins)
    print("average pt" , np.mean(particles))

interactive_main_wbroad = interactive(main_wbroad, nev=(0,1000,100), qhat=(1,3,0.5), L=(4,8,1))
interactive_main_wbroad

interactive(children=(IntSlider(value=500, description='nev', max=1000, step=100), FloatSlider(value=2.0, desc…